In [1]:
%pip install pygame

In [2]:
%pip install PyOpenGL PyOpenGL_accelerate

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GL.shaders import compileProgram, compileShader
import numpy as np
import random
import time


pygame 2.6.1 (SDL 2.28.4, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
# Initialize the screen
pygame.init()
screen = pygame.display.set_mode((800, 600), DOUBLEBUF | OPENGL)
pygame.display.set_caption("Fireworks Particle System")
clock = pygame.time.Clock()

In [5]:
# Vertex Shader
VERTEX_SHADER = """
#version 330 core
layout(location = 0) in vec2 position;
layout(location = 1) in vec4 color;
out vec4 vColor;
void main() {
    gl_Position = vec4(position, 0.0, 1.0);
    gl_PointSize = 10.0;
    vColor = color;
}
"""

# Fragment Shader
FRAGMENT_SHADER = """
#version 330 core
in vec4 vColor;
out vec4 FragColor;
void main() {
    float dist = length(gl_PointCoord - vec2(0.5));
    if (dist > 0.5) discard;
    FragColor = vColor;
}
"""

In [6]:
# Initialize Pygame and OpenGL
pygame.init()
screen = pygame.display.set_mode((800, 600), DOUBLEBUF | OPENGL)
pygame.display.set_caption("Animated Particle System")


In [7]:
glEnable(GL_BLEND)
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)

shader = compileProgram(
    compileShader(VERTEX_SHADER, GL_VERTEX_SHADER),
    compileShader(FRAGMENT_SHADER, GL_FRAGMENT_SHADER)
)

In [8]:
NUM_PARTICLES = 200
positions = np.zeros((NUM_PARTICLES, 2), dtype=np.float32)
velocities = np.random.uniform(-0.002, 0.002, (NUM_PARTICLES, 2)).astype(np.float32)
colors = np.random.rand(NUM_PARTICLES, 4).astype(np.float32)
colors[:, 3] = 1.0  # Full alpha

In [9]:
vao = glGenVertexArrays(1)
vbo_pos = glGenBuffers(1)
vbo_col = glGenBuffers(1)

glBindVertexArray(vao)

In [10]:
# Position buffer
glBindBuffer(GL_ARRAY_BUFFER, vbo_pos)
glBufferData(GL_ARRAY_BUFFER, positions.nbytes, positions, GL_DYNAMIC_DRAW)
glEnableVertexAttribArray(0)
glVertexAttribPointer(0, 2, GL_FLOAT, GL_FALSE, 0, None)


In [11]:
# Color buffer
glBindBuffer(GL_ARRAY_BUFFER, vbo_col)
glBufferData(GL_ARRAY_BUFFER, colors.nbytes, colors, GL_STATIC_DRAW)
glEnableVertexAttribArray(1)
glVertexAttribPointer(1, 4, GL_FLOAT, GL_FALSE, 0, None)

glBindBuffer(GL_ARRAY_BUFFER, 0)
glBindVertexArray(0)

In [12]:
clock = pygame.time.Clock()
running = True

while running:
    for event in pygame.event.get():
        if event.type == QUIT:
            running = False

    positions += velocities

    glBindBuffer(GL_ARRAY_BUFFER, vbo_pos)
    glBufferSubData(GL_ARRAY_BUFFER, 0, positions.nbytes, positions)

    glClearColor(0, 0, 0, 1)
    glClear(GL_COLOR_BUFFER_BIT)

    glUseProgram(shader)
    glBindVertexArray(vao)
    glDrawArrays(GL_POINTS, 0, NUM_PARTICLES)
    glBindVertexArray(0)
    glUseProgram(0)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()